# Introduction to Cross-Validation - Lab

## Introduction

In this lab, you'll be able to practice your cross-validation skills!


## Objectives

You will be able to:

- Perform cross validation on a model to determine optimal model performance
- Compare training and testing errors to determine if model is over or underfitting

## Let's get started

We included the code to pre-process below.

In [68]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

ames = pd.read_csv('ames.csv')

continuous = ['LotArea', '1stFlrSF', 'GrLivArea', 'SalePrice']
categoricals = ['BldgType', 'KitchenQual', 'SaleType', 'MSZoning', 'Street', 'Neighborhood']

ames_cont = ames[continuous]

# log features
log_names = [f'{column}_log' for column in ames_cont.columns]

ames_log = np.log(ames_cont)
ames_log.columns = log_names

# normalize (subract mean and divide by std)

def normalize(feature):
    return (feature - feature.mean()) / feature.std()

ames_log_norm = ames_log.apply(normalize)

# one hot encode categoricals
ames_ohe = pd.get_dummies(ames[categoricals], prefix=categoricals, drop_first=True)

preprocessed = pd.concat([ames_log_norm, ames_ohe], axis=1)

X = preprocessed.drop('SalePrice_log', axis=1)
y = preprocessed['SalePrice_log']

### Train-test split

Perform a train-test split with a test set of 20%.

In [69]:
# Import train_test_split from sklearn.model_selection
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression


In [70]:
# Split the data into training and test sets (assign 20% to test set)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


In [71]:
# A brief preview of train-test split
print(len(X_train), len(X_test), len(y_train), len(y_test))


1168 292 1168 292


### Fit the model

Fit a linear regression model and apply the model to make predictions on test set

In [72]:
# Your code here
linreg = LinearRegression()
linreg.fit(X_train, y_train)
y_hat_train = linreg.predict(X_train)
y_hat_test = linreg.predict(X_test)

### Residuals and MSE

Calculate the residuals and the mean squared error on the test set

In [73]:
# Your code here
train_err = mean_squared_error(y_train, y_hat_train)
test_err = mean_squared_error(y_test, y_hat_test)

print(train_err, test_err)

0.16334866357458117 0.16067552749515954


## Cross-Validation: let's build it from scratch!

### Create a cross-validation function

Write a function `kfolds()` that splits a dataset into k evenly sized pieces. If the full dataset is not divisible by k, make the first few folds one larger then later ones.

We want the folds to be a list of subsets of data!

In [74]:
data_frame = pd.DataFrame(preprocessed)

In [75]:
data_frame.head()

,LotArea_log,1stFlrSF_log,GrLivArea_log,SalePrice_log,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,KitchenQual_Fa,KitchenQual_Gd,...,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker
0,-0.133185,-0.803295,0.529078,0.559876,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0.113403,0.418442,-0.381715,0.212692,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0.419917,-0.576363,0.659449,0.733795,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0.103311,-0.439137,0.541326,-0.437232,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0.878108,0.112229,1.281751,1.014303,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0


In [76]:
k=7

In [77]:
num_observation = len(data_frame)
raw = num_observation/k
fold_size = num_observation//k
leftover = num_observation%k

print(fold_size, leftover, raw, len(data_frame), fold_size*k)

208 4 208.57142857142858 1460 1456


In [78]:
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np

def kfolds(data, k):
    # Force data as pandas DataFrame
    data = pd.DataFrame(data)
    num_observations = len(data)
    fold_size = num_observations//k
    leftovers = num_observations%k    
    # add 1 to fold size to account for leftovers
    folds = []
    start = 0
    for fold_n in range(1, k+1):
        if fold_n <= leftovers:
            fold = data.iloc[start : start+fold_size+1]
            folds.append(fold)
            start += fold_size + 1
        else:
            fold = data.iloc[start : start+fold_size]
            folds.append(fold)
            start += fold_size
            
    return folds

### Apply it to the Ames Housing data

In [79]:
# Make sure to concatenate the data again
ames_data = pd.concat([X.reset_index(drop=True), y], axis=1)

In [80]:
ames_data.describe()

,LotArea_log,1stFlrSF_log,GrLivArea_log,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,...,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,SalePrice_log
count,1.460000e+03,1.460000e+03,1.460000e+03,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,...,1460.00000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1.460000e+03
mean,-2.377326e-14,2.645829e-15,3.878937e-16,0.021233,0.035616,0.029452,0.078082,0.026712,0.401370,0.503425,...,0.05274,0.077397,0.017123,0.050685,0.040411,0.058904,0.017123,0.026027,0.007534,2.477124e-14
std,1.000000e+00,1.000000e+00,1.000000e+00,0.144209,0.185395,0.169128,0.268393,0.161297,0.490344,0.500160,...,0.22359,0.267312,0.129775,0.219429,0.196989,0.235526,0.129775,0.159271,0.086502,1.000000e+00
min,-3.750557e+00,-3.765335e+00,-4.367056e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-3.914887e+00
25%,-3.499336e-01,-7.091219e-01,-7.142678e-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-6.232379e-01
50%,8.878785e-02,-5.138002e-02,6.341863e-02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-5.644090e-02
75%,4.793745e-01,7.253213e-01,6.438801e-01,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.250575e-01
max,6.123695e+00,4.551387e+00,4.107983e+00,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,3.781237e+00


In [81]:
# Apply kfolds() to ames_data with 5 folds
folds = kfolds(ames_data,5)
folds[3].SalePrice_log

876    -0.579798
877     1.856638
878    -0.298117
879    -0.500614
880    -0.150331
          ...   
1163   -1.064769
1164    0.379426
1165    0.839831
1166    0.967301
1167    0.092617
Name: SalePrice_log, Length: 292, dtype: float64

### Perform a linear regression for each fold and calculate the training and test error

Perform linear regression on each and calculate the training and test error: 

In [82]:
test_errs = []
train_errs = []
k=5
linreg = LinearRegression()

for n in range(k):
    # Split in train and test for the fold
    train = pd.concat([fold for i, fold in enumerate(folds) if i != n])
    test = folds[n]
    
    # Redifine our test and train X, y
    X_train = train.drop('SalePrice_log', axis=1)
    y_train = train['SalePrice_log']
    
    X_test = test.drop('SalePrice_log', axis=1)
    y_test = test['SalePrice_log']
    
    # Fit a linear regression model
    linreg.fit(X_train, y_train)
    y_hat_train = linreg.predict(X_train)
    y_hat_test = linreg.predict(X_test)
    
    # Evaluate Train and Test errors
    test_error = mean_squared_error(y_test, y_hat_test)
    train_error = mean_squared_error(y_train, y_hat_train)
    
    test_errs.append(test_error)
    train_errs.append(train_error)

print(train_errs)
print(test_errs)

[0.17170509651464633, 0.1550793568593051, 0.15659946326223276, 0.1613455766630869, 0.15165048553131652]
[0.12431546148437402, 0.19350064631313094, 0.1891053043131114, 0.17079325250026892, 0.2074270458891691]


## Cross-Validation using Scikit-Learn

This was a bit of work! Now, let's perform 5-fold cross-validation to get the mean squared error through scikit-learn. Let's have a look at the five individual MSEs and explain what's going on.

In [83]:
# Your code here
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer

mse= make_scorer(mean_squared_error)

cv_5_results = cross_val_score(linreg, X, y, cv=5, scoring=mse)

Next, calculate the mean of the MSE over the 5 cross-validation and compare and contrast with the result from the train-test split case.

In [85]:
# Your code here
np.mean(cv_5_results), np.mean(train_errs), np.mean(test_errs)


(0.17702834210001087, 0.15927599576611753, 0.17702834210001087)

##  Summary 

Congratulations! You are now familiar with cross-validation and know how to use `cross_val_score()`. Remember that the results obtained from cross-validation are robust and always use it whenever possible! 